In [58]:
import numpy as np
from sklearn.datasets import  load_iris
data = load_iris()['data']

label = load_iris()['target']
label[:2]
td = data[label != 1]
la = label[label != 1] - 1

# sgd SVM

In [59]:
def pegasos(da, la, alpha=0.001, k=50, maxiters=100):
    m, n = da.shape
    w = np.random.randn(n, 1) 
    if k >= m:
        raise ValueError("kkkkk")
    iters = 0
    while iters < maxiters:
        iters += 1
        index = np.random.permutation(m)
        temp_data = da[index[:k]]
        temp_la = la[index[:k]].reshape(-1, 1)
        # 随机采样 然后进行计算 pred。
        pred = np.multiply(np.matmul(temp_data, w), temp_la)
        #print(pred.shape)
        
        t_d = []
        for i in range(k):
            # 对于误分的样本， 进行 w的累加。
            if pred[i] < 1:
                t_d.append(temp_data[i].tolist())
        td_temp_la = temp_la[pred<1].reshape(-1, 1)
        
        w_delta = np.matmul(np.array(t_d).T, td_temp_la)
        w = np.multiply((1 - 1 / iters), w) + np.multiply(alpha * (1 / k), w_delta)
    return w


def get_acc(test_data, test_la, w):
    pred_old = np.matmul(test_data, w)    
    pred_new = [[1] if i>=0.5 else [-1] for i in pred_old]
    acc = np.sum(pred_new == test_la.reshape((-1, 1)))
    lens = len(test_la)
    print(acc, lens)
    return acc / lens, pred_new, pred_old


w = pegasos(td, la)
#acc, pred = get_acc(td[40:70], la[40:70], w)
acc, pred, pred_old = get_acc(td, la, w)
acc

100 100


1.0

In [61]:
data = load_iris()['data']
label = load_iris()['target']
    # print(label)
data = data[label != 2]
label = label[label != 2]
#label = label - 1
#
label = np.array([-1 if i==0 else 1 for i in label]).reshape((-1, 1))

In [62]:
label = label.reshape(-1)
label

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1])

In [63]:
w = pegasos(data, label)
#acc, pred = get_acc(td[40:70], la[40:70], w)
acc, pred, pred_old = get_acc(data, label, w)
acc

59 100


0.59

In [5]:
# 单样本计算
def get_w(data, w):
    return np.matmul(data, w)

alpha = 0.001
m ,n = td.shape
w = np.random.randn(n, 1)
la = la.reshape(m, 1)

k = int(0.4 * m)
iters = 0
while iters < 100:
    iters += 1
    
    w_delta = np.zeros((n, 1))
    for i in range(k):
        temp = get_w(td[i], w)
        if np.multiply(la[i], temp) < 1:
            w_delta += np.matmul(td[i].reshape(n, 1), la[i]).reshape(n, 1)
    w = (1 - 1/iters) * w + alpha * ( 1 / k) * w_delta
w

array([[-0.13439842],
       [-0.0909307 ],
       [-0.03972065],
       [-0.00580105]])

In [6]:
# 矩阵运算。
    
iters = 1
while iters < 100:
    iters += 1
    index = np.random.permutation(m)
    temp_data = td[index[:k]]
    temp_la = la[index[:k]]
    pred = np.multiply(get_w(temp_data, w), temp_la)
    
    t_d = []
    for i in range(k):
        if pred[i] < 1:
            t_d.append(temp_data[i].tolist())
    td_temp_la = temp_la[pred<1].reshape(-1,1)
    
    w_delta = np.matmul(np.array(t_d).T, td_temp_la)
    w = np.multiply((1 - 1 / iters), w) + np.multiply(alpha * (1 / k), w_delta)
w

array([[ 0.03692377],
       [-0.01302973],
       [ 0.10170057],
       [ 0.04463459]])

# SMO算法 计算

In [7]:
from numpy import *

In [8]:
def loaddataset(filename):
    datamat =[];label=[]
    fr = open(filename)
    for line in fr.readlines():
        linearr = line.strip().split('\t')
        datamat.append([float(linearr[0]),float(linearr[1])])
        label.append(float(linearr[2]))
    return datamat,label

In [9]:
data,label = loaddataset(path)

NameError: name 'path' is not defined

In [10]:
def selectJrand(i,m):
    if i>m:
        #raise ValueError("The i: %d must smaller than m"%(i))
        raise ValueError("The i: {} must smaller than m".format(i))
    j = i
    while j==i:
        j = int(np.random.uniform(0,m))
    return j

def clipAlpha(aj,H,L):
    if aj > H:
        aj = H
    elif aj < L:
        aj = L
    else:
        pass
    return aj

In [11]:
def smosimple(dataMatIn,classLabels,C,toler,MaxIter):
    dataMatrix = mat(dataMatIn);
    labelMat = mat(classLabels).transpose()
    b = 0
    m,n=shape(dataMatIn)
    alphas = mat(zeros((m,1)))
    iters =0
    while (iters<MaxIter):
        alphaPairsChanged = 0
        for i in range(m):
            fxi = float(np.multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[i,:].T))+b
            Ei = fxi - float(labelMat[i])
            if((labelMat[i]*Ei<-toler) and (alphas[i]<C)) or ((labelMat[i]*Ei>toler and (alphas[i]<C))):
                j = selectJrand(i,m)
                fxj = float(np.multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[j,:].T))+b
                Ej =fxj-float(labelMat[j])
               
            # 复制 以备用。
                alphaIold = alphas[i].copy()
                alphaJold = alphas[j].copy()
                                             
                if (labelMat[i]!=labelMat[j]):
                    L = max(0,alphas[j]-alphas[i])
                    H = min(C,C+alphas[j]-alphas[i])
                else:
                    L = max(0,alphas[j]+alphas[i]-C)
                    H = min(C,alphas[j]+alphas[i])
                if L==H:
                    print("L==H")
                    continue
               
            # 计算eta 用来更新alpha
                eta = 2.0 * dataMatrix[i,:]*dataMatrix[j,:].T-dataMatrix[i,:]*dataMatrix[i,:].T-\
                                                             dataMatrix[j,:] * dataMatrix[j,:].T
               
                if eta>=0:
                    print("eta>=0")
                    continue
               
            # 更新 alphas [j]
                alphas[j] -= labelMat[j] *(Ei-Ej)/eta
                alphas[j] = clipAlpha(alphas[j],H,L)
               
                if (abs(alphas[j]-alphaJold)<0.00001):
                    print("j not moving enough")
                    continue
                   
            # 更新 alphas [i]
                alphas[i] += labelMat[j]*labelMat[i]*(alphaJold-alphas[j])
               
            # 更新 b 两个  不满足条件 就求均值。
                b1 = b - Ei - labelMat[i] * (alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[i,:].T-\
                labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[i,:]*dataMatrix[j, :].T
               
                b2 = b - Ej - labelMat[i] * (alphas[i]-alphaIold)*dataMatrix[i, :]*dataMatrix[j,:].T-\
                labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[j, :]*dataMatrix[j, :].T
               
                if (0<alphas[i]) and (C>alphas[i]):
                    b = b1
                elif (0<alphas[j]) and (C>alphas[j]):
                    b = b2
                else:
                    b = (b1+b2)/2.
                
                alphaPairsChanged +=1
                print("iterL%d i :%d,pairs changed %d" %(iters,i,alphaPairsChanged))
               
        if (alphaPairsChanged==0):
            iters +=1
        else:
            iters =0
        print("iteration number :%d" % iters)
    return b,alphas           

In [12]:
"""
label = []
data = []
with open(path,'r') as f:
    line = f.readline().strip().split('\t')
    print(line)
    while line:
        temp = [i for i in line[:2]]
        temp_label = [i for i in line[2:3]]
        data.append(temp)
        label.append(temp_label)
        line = f.readline().strip().split('\t')
"""

"\nlabel = []\ndata = []\nwith open(path,'r') as f:\n    line = f.readline().strip().split('\t')\n    print(line)\n    while line:\n        temp = [i for i in line[:2]]\n        temp_label = [i for i in line[2:3]]\n        data.append(temp)\n        label.append(temp_label)\n        line = f.readline().strip().split('\t')\n"

In [13]:
# b,alphas = smosimple(data,label,0.6,0.001,40)
